In [1]:
%matplotlib inline
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
from astropy.utils.data import get_pkg_data_filename

In [2]:
#start of bruno's code

import subprocess as sp

import glob

import astropy.io.fits as pyfits



def scampparams():

    pars = ["XWIN_IMAGE", "YWIN_IMAGE", "ERRAWIN_IMAGE", "ERRBWIN_IMAGE",\

            "ERRTHETAWIN_IMAGE", "FLUX_AUTO", "FLUXERR_AUTO", "FLAGS",\

            "FLAGS_WEIGHT", "FLUX_RADIUS", "MAG_AUTO","ALPHA_J2000","DELTA_J2000"]



    f = open("scamp.param","w")

    for p in pars:

        f.write(f"{p}\n")

    f.close()

    return None


# SANITY check of Zeropoint calculations
#### note that the below test example is just a weird one?  header keywords & values arent there for some reason... ...maybe b/c this was a "total" file?

In [3]:
#looking at hst fits data to get values needed for remaining zeropt calculations (my code)--(test1_image)

#verifying calculation matches known value from bruno
test1_file = fits.open('/data1/rowland/fits_data/hst_14096_i1_acs_wfc_total_drz.fits') #uses acs f435w
#test_file.info #shows theres 1 header, 4 images, 1 BinTable images - well [1] should be the SCI image then


test1_header0 = test1_file[0].header #header
''' 
test1_header1 = test1_file[1].header #science image 'SCI'
test1_header2 = test1_file[2].header #weights image 'WHT'
test1_header3 = test1_file[3].header #something we wont use image 'CTX'
test1_header4 = test1_file[4].header #wait where did this come from 'EXP'
test1_header5 = test1_file[5].header #whatever a bintable is 'HDRTAB' think it formats header list
''' 
#print(test1_header1['PHOTFLAM'])

#test1_file[0].header #goes from 0-5
#test1_header0["FILTER"] #NOT FOUND
test1_header0["FILTER2"] #returns 'F435W'


'F435W'

In [4]:
#trying another file (test2 image) b/c maybe its just that one? ------that one IS just weird....--test2 image
#verifying calculation matches known value from bruno
test2_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f814w_drz.fits') #uses acs f435w
#test_file.info #shows theres 1 header, 4 images, 1 BinTable images - well [1] should be the SCI image then

test2_file[0].header #goes from 0-4

test2_header0 = test2_file[0].header #header
test2_header1 = test2_file[1].header #science image 'SCI'
test2_header2 = test2_file[2].header #weights image 'WHT'
test2_header3 = test2_file[3].header #something we wont use image 'CTX'
test2_header4 = test2_file[4].header #wait where did this come from 'EXP'
 
#print('PHOTFLAM:',test2_header1['PHOTFLAM'])
#print('PHOTPLAM:',test2_header1['PHOTPLAM'])

PHOTFLAM_test2 = test2_header1['PHOTFLAM'] #7.0073061E-20 / inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_test2 = test2_header1['PHOTPLAM'] #8.0593491E+03 / Pivot wavelength (Angstroms)
#test2_header0["FILTER"] #NOT FOUND
#print('FILTER2:',test2_header0["FILTER2"]) #returns 'F814W'
#print('FILTER1:',test2_header0['FILTER1']) #returns 'CLEAR1L'

In [5]:
#ZEROpoint calculation for test2

ZP_f814w = (-2.5 * np.log10(PHOTFLAM_test2)) - (5 * np.log10(PHOTPLAM_test2)) - 2.408
print(ZP_f814w, 'should match expected value of:', "f814w:25.97")
print('this is very very close, but not 100% the same - ASK BRUNO!')

25.94662243630965 should match expected value of: f814w:25.97
this is very very close, but not 100% the same - ASK BRUNO!


# NOTE TO SELF: 
### [for trying to automate selections, idk why but filter changes from 1 to 2 for correct name (see above test2 & below f850lp for an example)]

# Calculating ZEROpoints of remaining filters

In [6]:
#calculating remaining 3 zeropoints (f850lp, f775w, f625w)


#first f850lp
f850lp_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f850lp_drz.fits')
f850lp_header0 = f850lp_file[0].header #header

f850lp_header1 = f850lp_file[1].header #science image 'SCI'
f850lp_header2 = f850lp_file[2].header #weights image 'WHT'
f850lp_header3 = f850lp_file[3].header #something we wont use image 'CTX'
f850lp_header4 = f850lp_file[4].header #wait where did this come from 'EXP'
#f850lp_header5 = f850lp_file[5].header #whatever a bintable is 'HDRTAB' think it formats header list

#print(f850lp_header1['PHOTFLAM']) 
#f850lp_file.info
#f850lp_file[0].header #goes from 0-4
#f850lp_header0["FILTER"] #NOT FOUND
#print(f850lp_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f850lp_header0["FILTER1"])#returns 'F850LP'

#needed values
PHOTFLAM_f850lp = f850lp_header1['PHOTFLAM']#1.5213061E-19 #/ inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_f850lp = f850lp_header1['PHOTPLAM']#9.0336270E+03 #/ Pivot wavelength (Angstroms)   

FILTER1: F850LP


In [7]:
#second f775w

f775w_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f775w_drz.fits')

#f775w_file.info
#f775w_file[0].header #goes from 0-4

f775w_header0 = f775w_file[0].header #header
f775w_header1 = f775w_file[1].header #science image 'SCI'
f775w_header2 = f775w_file[2].header #weights image 'WHT'
f775w_header3 = f775w_file[3].header #something we wont use image 'CTX'
f775w_header4 = f775w_file[4].header #wait where did this come from 'EXP'

##print('PHOTFLAM:',f775w_header1['PHOTFLAM']) 
#f775w_header1

#print('FILTER:',f775w_header0["FILTER"]) #NOT FOUND
print('FILTER2:',f775w_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f775w_header0["FILTER1"])#returns 'F775W'

#needed values
PHOTFLAM_f775w = f775w_header1['PHOTFLAM']#9.9942165E-20 / inverse sensitivity, ergs/cm2/Ang/electron
PHOTPLAM_f775w = f775w_header1['PHOTPLAM']#7.6936714E+03 / Pivot wavelength (Angstroms) 

FILTER2: CLEAR2L
FILTER1: F775W


In [8]:
#third f625w

f625w_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f625w_drz.fits')

#f625w_file.info
###f625w_file[4].header #goes from 0-4

f625w_header0 = f625w_file[0].header #header
f625w_header1 = f625w_file[1].header #science image 'SCI'
f625w_header2 = f625w_file[2].header #weights image 'WHT'
f625w_header3 = f625w_file[3].header #something we wont use image 'CTX'
f625w_header4 = f625w_file[4].header #wait where did this come from 'EXP'

print('PHOTFLAM:',f625w_header1['PHOTFLAM']) 
print('PHOTPLAM:',f625w_header1['PHOTPLAM'])
#f625w_header1

#print('FILTER:',f625w_header0["FILTER"]) #NOT FOUND
print('FILTER2:',f625w_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f625w_header0["FILTER1"])#returns 'F625W'

#needed values
PHOTFLAM_f625w = f625w_header1['PHOTFLAM']#1.1931234E-19 / inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_f625w = f625w_header1['PHOTPLAM']#6.3120513E+03 / Pivot wavelength (Angstroms) 

PHOTFLAM: 1.1931234e-19
PHOTPLAM: 6312.0513
FILTER2: CLEAR2L
FILTER1: F625W


In [9]:
photflam_list = [PHOTFLAM_f850lp, PHOTFLAM_f775w, PHOTFLAM_f625w]
photplam_list = [PHOTPLAM_f850lp, PHOTPLAM_f775w, PHOTPLAM_f625w]
#phot_list = [(PHOTFLAM_test2, PHOTPLAM_test2)] #tested for proof of concept - worked!
phot_list = [(PHOTFLAM_f850lp, PHOTPLAM_f850lp), (PHOTFLAM_f775w, PHOTPLAM_f775w), (PHOTFLAM_f625w, PHOTPLAM_f625w)]
ZPs = []
for x,y in phot_list:
    Zeropts = (-2.5 * np.log10(x)) - (5 * np.log10(y)) - 2.408
    ZPs.append(Zeropts)
ZPs = np.array(ZPs)
print('in the order of f850lp, f775w, f625w, the zeropts are:',ZPs)

in the order of f850lp, f775w, f625w, the zeropts are: [24.85714771 25.66195995 25.89943399]


# Making dictionaries with all filter values included

In [10]:
#creating needed dictionaries with calculated zeropoints included

#need a dictionary for zeropoint values of each filter
zeropoints = {"f435":25.665,"f606w":26.493,"f814w":25.97,"f105w":26.2687,"f125w":26.2303,"f140w":24.4524,"f160w":25.9463, "f625w":24.85714771, "f775w":25.66195995, "f850lp":25.89943399}

#need to create a dictionary for the 10 indiviual filters
filterImages = {"f435w":'F435W',"f606w":'F606W',"f814w":'F814W',"f625w":'F625W',"f775w":'F775W',"f850lp":'F850LP',"f105w":'F105W',"f125w":'F125W',"f140w":'F140W',"f160w":'F160W'}

# SANITY  check of splitting 'sci' & 'wht' frames

In [11]:
#testing what values for loop gathers --test3 image

test3_image = fits.open('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits') # equal to 'hdu' in for loop

test3_header = test3_image[0].header 
test3_filename = test3_header["FILTER"] #so this will work for SE files, prob not NW ones tho
print('test3_filename:',test3_filename) #seeing if this worked
test3_gain = float(test3_image[0].header["CCDGAIN"])
print('test3_gain:',test3_gain) #not sure what this means....
test3_exptime = float(test3_image[0].header['EXPTIME'])
print('test3_exptime:',test3_exptime) #exposure time
#other stuff in code
test3_headerOut = test3_image["SCI"].header
#this part builds a header from available info if filter name doesnt match list
    #print(test_headerOut) #prints the science frame header for each file
    #lines after that pull values from frame 0 to add to science frame header
    #outname uses science frame location as name of 
test3_dataOUt = test3_image["SCI"].data/test3_gain
#print(test_dataOUt) #wtf is this?
#for i in test_dataOUt: 
#    print(i)
print('shape of test3_dataOUt:',np.shape(test3_dataOUt)) #(2069, 2124)
print('shape of test3_image.data:',np.shape(test3_image["SCI"].data)) #(2069,2124) (#arrays,len of each array) #all the data is 0 matrices tho
test3_image['SCI'].data[0]

test3_filename: F160W
test3_gain: 2.5
test3_exptime: 1005.872924
shape of test3_dataOUt: (2069, 2124)
shape of test3_image.data: (2069, 2124)


array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [12]:
#figured out error - below code splits & rewrites files into original & sci & sci.wht files--test4 image 

#why getting no "CCDGAIN" errors now
#somehow (overwrite below?) some filenames changed to "_sci.wht.fits" to end of filenames
test4_SEfile = fits.open('/data1/rowland/fits_data/fits_data_Doubles_SE/hst_14096_j2_wfc3_ir_f125w_drz_SE.fits')
#test_SEfile[0].header
test4_SEfile.info

<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f713951d320>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f7139535b00>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f7139543978>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f7139549c50>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f71395048d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f713950fc50>]>

# Splitting 'sci' & 'wht' files

In [13]:
# need to split 3 files from hst fits data files for sextractor intake reasons
# CAN ONLY RUN THIS ONCE B/C IT REWRITES FILES AFTER SEPARATING! will get error if done more than once

#only doing on doubles/SE files first to test on smaller dataset and its in a different location anyway
all_SE_files = glob.glob('/data1/rowland/fits_data/fits_data_Doubles_SE/hst*.fits')

for image in all_SE_files:

    hdu = pyfits.open(image)

    header = hdu[0].header

    filterName = header["FILTER"] #not sure this will work given above "NOTE TO SELF"

    gain = float(hdu[0].header["CCDGAIN"])

    exptime = float(hdu[0].header["EXPTIME"])



    if filterName not in filterImages.keys():

        filterImages[filterName] = []



    print("%s_sci.fits"%(image[:-5])) #thats the output below of sci file locations

    headerOut = hdu["SCI"].header

    headerOut["FILTER"] = filterName

    headerOut["EXPTIME"] = exptime

    headerOut["GAIN"] = gain



    outname = "%s_sci.fits"%(image[:-5])

    dataOUt = hdu["SCI"].data/gain

    pyfits.writeto(outname,dataOUt,header=headerOut,overwrite=True) #b/c overwrite cant repeat this cell...

    pyfits.writeto("%s_sci.wht.fits"%(image[:-5]),hdu["WHT"].data,header=hdu["WHT"].header,overwrite=True)



    filterImages[filterName].append(outname)

    # break

print('FILTER IMAGES printout:',filterImages)

KeyError: "Keyword 'CCDGAIN' not found."

In [15]:
print('FILTER IMAGES printout:',filterImages)

FILTER IMAGES printout: {'f435w': 'F435W', 'f606w': 'F606W', 'f814w': 'F814W', 'f625w': 'F625W', 'f775w': 'F775W', 'f850lp': 'F850LP', 'f105w': 'F105W', 'f125w': 'F125W', 'f140w': 'F140W', 'f160w': 'F160W'}


# SANITY check SExtractor start

In [16]:
### test 3 START
#### might have found error : maybe filterImages isn't the list of filters????


#testing below SExtractor run by first getting single file data in similar form---test3 image reused

single_test3_file = glob.glob('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits')
test3_filterImages = {'f160w':'F160W'} #short dic

for image in single_test3_file:
    #gathers header info we want

    hdu3 = pyfits.open(image)

    header3 = hdu3[0].header

    filterName3 = header3["FILTER"] #not sure this will work given above "NOTE TO SELF"

    gain3 = float(hdu3[0].header["CCDGAIN"])

    exptime3 = float(hdu3[0].header["EXPTIME"])


    #saves header info we want to each file (sci, wht) after separating out from original compressed file 
    if filterName3 not in test3_filterImages.keys():

        test3_filterImages[filterName3] = []



    print("%s_sci.fits"%(image[:-5])) #thats the output below of sci file locations

    headerOut3 = hdu3["SCI"].header

    headerOut3["FILTER"] = filterName3

    headerOut3["EXPTIME"] = exptime3

    headerOut3["GAIN"] = gain3



    outname3 = "%s_sci.TEST3.fits"%(image[:-5])

    dataOUt3 = hdu["SCI"].data/gain3

    pyfits.writeto(outname3,dataOUt3,header=headerOut3,overwrite=True) #b/c overwrite cant repeat this cell...

    pyfits.writeto("%s_sci.wht.TEST3.fits"%(image[:-5]),hdu3["WHT"].data,header=hdu3["WHT"].header,overwrite=True)



    test3_filterImages[filterName3].append(outname3)

    # break

print('test3_FILTER_IMAGES printout:',test3_filterImages) #good prints out single file so i can test below w/o crashing cpu

/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
test3_FILTER_IMAGES printout: {'f160w': 'F160W', 'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}


In [17]:
#test3_filterImages[filterName3] #prints ['filename.fits']
test3_filterImages

{'f160w': 'F160W',
 'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}

In [18]:
#running test3 image outside of the for loop below so can compare anwsers

test3_thresh = 20.0
test3_sci = fits.open('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits')#_sci.TEST3.fits')

#simple entries based on gal.fit example with SEx

#these gives errors
#! sex -PARAMETERS_NAME default.param hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits -DETECT_THRESH {test3_thresh} #error opening file
#! sex -PARAMETERS_NAME default.param {test3_sci} -DETECT_THRESH {test3_thresh} #errors finding file

#this way works
! sex -PARAMETERS_NAME default.param /data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits -DETECT_THRESH {test3_thresh} #this one works!

> 
----- SExtractor 2.19.5 started on 2019-06-12 at 11:48:19 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at lin

In [19]:
#trying above as a cycle now so for loop will work

test3_sci_cycle = ('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits')
                  #/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3
! sex -PARAMETERS_NAME default.param {test3_sci_cycle} -DETECT_THRESH {test3_thresh} #works!

> 
----- SExtractor 2.19.5 started on 2019-06-12 at 11:48:20 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at lin

In [20]:
#trying above again but trying to save outputs

test3_SEoutput = ! sex -PARAMETERS_NAME default.param {test3_sci_cycle} -DETECT_THRESH {test3_thresh} 
#print('this is my printout:', test3_SEoutput) #weird format of print out but it did work

In [21]:
#testing below on single file first --test3 image reused
test3_thresh = 20.0

test3_skycat ="SDSS-R9"

for filter in test3_filterImages.keys(): #keys come after: (name:keys)

    test3_catalogs = ""

    for image in test3_filterImages[filter]:
        #print('image:', image)#?? gave 5 iterations to print F160W separately...weird

        test3_root = (image[:-5]) #this is the "..._sci.TEST3" file that was written during above for loop, not sure why the [:-5] tho
        print('this is test3_root:',test3_root) #ERROR B/C DOESNT END WITH '.FITS'
        
        ####adding .fits to end so !sex will work......alternatively just use image!!! 
        ####test3_fn = "%s.fits"%(image[:-5])
        ####print('hope this works:',test3_fn)
        
        #not sure how this gets inputted, maybe add to ! sex -PARAMETERS_NAME....line?
        test3_catalogs += f" {test3_root}.cat" #possibly CATALOG_NAME parameter using but why 5 printouts?
        #print(test3_catalogs) #??? no clue, just prints 5 rows of '.cat', with increasing array lengths 
        
        ####! sex -PARAMETERS_NAME default.param {test3_fn} -DETECT_THRESH {test3_thresh} #this works now!
        ! sex -PARAMETERS_NAME default.param {image} -DETECT_THRESH {test3_thresh} #this works now!


this is test3_root: 
> 
----- SExtractor 2.19.5 started on 2019-06-12 at 11:48:22 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights

> *Error*: cannot open F

this is test3_root: 
> 
----- SExtractor 2.19.5 started on 2019-06-12 at 11:48:22 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights

> *Error*: cannot open 1

this is test3_root: 
> 
----- SExtractor 2.19.5 started on 2019-06-12 at 11:48:22 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights

> *Error*: cannot open 6

this is test3_root: 
> 
----- SExtractor 2.19.5 started on 2019-06-12 at 11:48:22 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights

> *Error*: cannot open 0

this is test3_root: 
> 
----

> Line: 1525  Objects:      170 detected /       16 sextracted
> Line: 1550  Objects:      175 detected /       18 sextracted
> Line: 1575  Objects:      178 detected /       22 sextracted
> Line: 1600  Objects:      180 detected /       23 sextracted
> Line: 1625  Objects:      182 detected /       26 sextracted
> Line: 1650  Objects:      183 detected /       30 sextracted
> Line: 1675  Objects:      185 detected /       31 sextracted
> Line: 1700  Objects:      186 detected /       37 sextracted
> Line: 1725  Objects:      187 detected /       39 sextracted
> Line: 1750  Objects:      187 detected /       46 sextracted
> Line: 1775  Objects:      187 detected /       54 sextracted
> Line: 1800  Objects:      188 detected /       58 sextracted
> Line: 1825  Objects:      189 detected /       61 sextracted
> Line: 1850  Objects:      189 detected /       67 sextracted
> Line: 1875  Objects:      189 detected /       68 sextracted
> Line: 1900  Objects:      189 detected /       73 sex

In [22]:
#exploring test3_filterImages to get a sense of why for loop does 5 iterations in below cell 


#print(test3_filterImages) #{'f160w': 'F160W', 'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}

#for i in test3_filterImages:
#    print(i) #(prints)f160w (next row) F160W

#type(test3_filterImages) #dict

#np.shape(test3_filterImages) #()

#len(test3_filterImages) #2

#test3_filterImages.header #dict so doesnt have, cant be a fits anymore

#test3_filterImages??

for image in test3_filterImages[filter]:
    print(image) #only single file pathway prints
    #print(image[:-5]) #file pathway w/o .fits (aka the last5 indices) obviously

/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits


# SExtractor start

In [23]:
# 2nd part of Bruno's code that actually runs sextractor AMENDED by me to run in jupyter nb

thresh = 20.0

skycat ="SDSS-R9"

for filter in filterImages.keys(): #keys come after: (name:keys)

    catalogs = ""

    for image in filterImages[filter]:

        root = image[:-5]

        catalogs += f" {root}.cat"

#dangerous line DONT RUN!    #test_commandSEx = !sex -{image} -DETECT_THRESH {thresh} -CATALOG_TYPE FITS_LDAC -CATALOG_NAME {root}.cat -CHECKIMAGE_TYPE NONE -PARAMETERS_NAME scamp.param -FILTER Y -FILTER_NAME gauss_1.5_3x3.conv -SEEING_FWHM 0.5 -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints} -ANALYSIS_THRESH {thresh} -DETECT_THRESH {thresh}"
       # print(test_commandSEx)

        #sp.call(commandSEx,shell=True)


In [24]:



commandSCAMP = f"scamp {catalogs} -ASTREF_CATALOG {skycat}"

print(commandSCAMP)

sp.call(commandSCAMP,shell=True)



command = "SWarp -IMAGEOUT_NAME %s_%s.fits -COMBINE_TYPE SUM -PIXELSCALE_TYPE MANUAL -PIXEL_SCALE 0.12 -GAIN_KEYWORD GAIN -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_SUFFIX .wht.fits %s"%("ElGordo", filter," ".join(filterImages[filter]))

print(command)

sp.call(command,shell=True)

scamp  .cat .cat .cat .cat .cat -ASTREF_CATALOG SDSS-R9
SWarp -IMAGEOUT_NAME ElGordo_f160w.fits -COMBINE_TYPE SUM -PIXELSCALE_TYPE MANUAL -PIXEL_SCALE 0.12 -GAIN_KEYWORD GAIN -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_SUFFIX .wht.fits F 1 6 0 W


127